In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
from scipy import stats
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import seaborn as sns
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Загружаем данные

In [ ]:
df = pd.read_csv("../input/kc-house-data/kc_house_data.csv")
df[df.isnull().any(axis=1)]
df.head()

#  Описание

In [ ]:
count_row = df.shape[0]  #Кол-во наблюдений (строк)
count_col = df.shape[1]  #КОл-во характеристик (переменных)
print("Количество наблюдений", count_row)
print("Количество характеристик", count_col)

In [ ]:
df.info()

In [ ]:
stats = df.describe()
stats=stats.transpose()
pd.DataFrame(stats)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df1=df[['price', 'bedrooms', 'bathrooms', 'sqft_living',
    'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
    'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
    'lat', 'long', 'sqft_living15', 'sqft_lot15']]
h = df1.hist(bins=25,figsize=(16,16),xlabelsize='10',ylabelsize='10',xrot=-15)
sns.despine(left=True, bottom=True)
[x.title.set_size(12) for x in h.ravel()];
[x.yaxis.tick_left() for x in h.ravel()];

In [ ]:
sns.set(style="whitegrid", font_scale=1)

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=df['bedrooms'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['floors'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='Bedrooms', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='Floors', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.despine(left=True, bottom=True)
sns.boxplot(x=df['bathrooms'],y=df['price'], ax=axe)
axe.yaxis.tick_left()
axe.set(xlabel='Bathrooms / Bedrooms', ylabel='Price');

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=df['waterfront'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['view'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='Waterfront', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='View', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.boxplot(x=df['grade'],y=df['price'], ax=axe)
sns.despine(left=True, bottom=True)
axe.yaxis.tick_left()
axe.set(xlabel='Grade', ylabel='Price');

In [ ]:
df_dm=df.copy()
df_dm.describe()

In [ ]:
#удаляем данные
df1 = df.drop(["zipcode","lat","long","id","date"], axis = 1)
df1.head()

In [ ]:
corr = df1.corr()
corr.style.background_gradient(cmap="coolwarm").set_precision(2)

sqft_living сильно коррелирует с sqft_above, а так как первый признак имеет большее влияние на цеелевой параметр, то оставим его. Точно такая же ситуация наблюдается между sqft_living и grade

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
X = df[['sqft_living']].values
y = df[['price']].values
regr = LinearRegression()

quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)
X_quad = quadratic.fit_transform(X)
X_cubic = cubic.fit_transform(X)

X_fit = np.arange(X.min(),X.max(),1)[:,np.newaxis]
regr = regr.fit(X,y)
y_lin_fit = regr.predict(X_fit)
linear_r2 = r2_score(y,regr.predict(X))

regr = regr.fit(X_quad,y)
y_quad_fit = regr.predict(quadratic.fit_transform(X_fit))
quadratic_r2 = r2_score(y,regr.predict(X_quad))

regr = regr.fit(X_cubic,y)
y_cubic_fit = regr.predict(cubic.fit_transform(X_fit))
cubic_r2 = r2_score(y,regr.predict(X_cubic))

plt.scatter(X,y,label='training points',color='lightgray')
plt.plot(X_fit,y_lin_fit,label='linear (d=1), $R^2=%.2f$' % linear_r2, color='blue', lw=2, linestyle=':')
plt.plot(X_fit,y_quad_fit,label='quadratic (d=2), $R^2=%.2f$' % quadratic_r2, color='red', lw=2, linestyle='-')
plt.plot(X_fit,y_cubic_fit,label='cubic (d=3), $R^2=%.2f$' % cubic_r2, color='green', lw=2, linestyle='--')
plt.xlabel('% lower status of the popelation [LSTAT]')
plt.ylabel('Price in $1000\'s [MEDV]')
plt.legend(loc='upper right')
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
#df1 = df.drop(["zipcode","lat","long","id","date","price"], axis = 1)
X = df1[['sqft_living']].values
#X = df1.values
y = df[['price']].values
    
from sklearn import tree
clf = tree.DecisionTreeRegressor(max_leaf_nodes=40)
clf = clf.fit(X, y)
tree_r2 = r2_score(y, clf.predict(X))
tree_r2

In [ ]:
X = df1[['sqft_living','bathrooms','sqft_living',
         'waterfront','grade',
         'yr_built']].values
#X = df1.values
y = df[['price']].values
    
from sklearn import tree
clf = tree.DecisionTreeRegressor(max_leaf_nodes=40)
clf = clf.fit(X, y)
tree_r2 = r2_score(y, clf.predict(X))
tree_r2